In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import constants
from core.tools.metrics import *
from core.models.stat_models import *
from core.models.baseline_rnn import *
from core.tools.visualize import *
from core.tools.time_series import *
from core.tools.data_import import *
import core.models.baseline_lstm as baseline_lstm

In [3]:
# Avaiable datasets, loaded from constants.
pprint(constants.DATA_DIR)
choice = None
while choice is None or choice not in constants.DATA_DIR.keys():
    if choice is not None:
        print("Invalid data location received, try again...")
    choice = input("Select Dataset >>> ")
FILE_DIR = constants.DATA_DIR[choice]

{'0': '/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv',
 '1': '/home/ec2-user/AnnEconForecast/data/UNRATE.csv',
 '2': '/home/ubuntu/AnnEconForecast/data/UNRATE.csv',
 '3': '/home/ec2-user/AnnEconForecast/data/DEXCAUS.csv'}
Select Dataset >>> 0


In [4]:
print(f"Dataset chosen: \n{FILE_DIR}")

Dataset chosen: 
/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv


In [5]:
# Pre-processing Parameters
PERIODS = 1
ORDER = 1
LAGS = 12
TRAIN_RATIO = 0.8

In [6]:
# Neural Network Parameters
NUM_TIME_STEPS = LAGS
# Number of series used to predict. (including concurrent)
NUM_INPUTS = 1
NUM_OUTPUTS = 1
NUM_NEURONS = 64
# Number of output series
LEARNING_RATE = 0.01
EPOCHS = 50
# Training Settings
REPORT_PERIODS = 1
TENSORBOARD_DIR = "./tensorboard/test/1"

In [12]:
parameters = {
    "epochs": 50,
    "num_time_steps": LAGS,
    "num_inputs": 1,
    "num_outputs": 1,
    "num_neurons": 64,
    "learning_rate": 0.01,
    "report_periods": 1,
    "tensorboard_dir": "./tensorboard/test/1"
}

8

In [7]:
prepared_df = baseline_lstm.prepare_dataset(
    file_dir=FILE_DIR,
    periods=PERIODS,
    order=ORDER
)

Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
First few rows of dataset loaded:
            UNRATE_period1_order1
DATE                             
1948-02-01                    0.4
1948-03-01                    0.2
1948-04-01                   -0.1
1948-05-01                   -0.4
1948-06-01                    0.1


In [8]:
# Normalize data
(X_train, X_val, X_test,
 y_train, y_val, y_test) = baseline_lstm.normalize(
    df=prepared_df,
    train_ratio=TRAIN_RATIO,
    lags=LAGS
)

Total 836 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (836, 12, 1), y shape = (836, 1, 1)
Training and testing set generated,        
X_train shape: (601, 12, 1)        
y_train shape: (601, 1)        
X_test shape: (168, 12, 1)        
y_test shape: (168, 1)        
X_validation shape: (67, 12, 1)        
y_validation shape: (67, 1)


In [9]:
data_collection = {
    "X_train": X_train,
    "X_val": X_val,
    "X_test": X_test,
    "y_train": y_train,
    "y_val": y_val,
    "y_test": y_test
}

In [10]:
# Run the model
baseline_lstm.exec_core(
    parameters=parameters,
    data_collection=data_collection,
    clip_grad=None
)

Resetting Tensorflow defalut graph...
Note: no gradient clipping is applied.            
If possible gradient exploding detected (e.g. nan loss), try use clip_grad.

Iteration [0], Training MSE 3.3688972; Validation MSE 1.3199933

Iteration [1], Training MSE 3.0712655; Validation MSE 0.8637946

Iteration [2], Training MSE 1.9280287; Validation MSE 0.5594504

Iteration [3], Training MSE 1.1122528; Validation MSE 0.4136369

Iteration [4], Training MSE 1.2073007; Validation MSE 0.6260026

Iteration [5], Training MSE 1.3842461; Validation MSE 0.7193491

Iteration [6], Training MSE 1.3110825; Validation MSE 0.5857255

Iteration [7], Training MSE 1.1198708; Validation MSE 0.4017738

Iteration [8], Training MSE 0.9975752; Validation MSE 0.3262978

Iteration [9], Training MSE 0.9960107; Validation MSE 0.3718212

Iteration [10], Training MSE 1.0303591; Validation MSE 0.4234388

Iteration [11], Training MSE 1.0297691; Validation MSE 0.4099090

Iteration [12], Training MSE 1.0083833; Validation M

{'mae': 0.5622803, 'mse': 0.5026706, 'rmse': 0.70899266, 'mape': 4.5396376}